Sript to clean the data from a currupted study and to create a temporaney study

In [76]:
import os

import optuna
from optuna.trial import FrozenTrial
from optuna.importance import get_param_importances
from optuna.visualization import plot_param_importances
import pandas as pd
import numpy as np
import datetime
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pio.templates.default = "plotly"

from settings.config import *
from src.training.commons import init_optuna_storage
from src.commons.exp_config import HTunerExpConfig

In [77]:
exp_dir = os.path.join(EXPERIMENTS_PATH, "basic")
exp_name = "resnets_htuning{25k}"
study_name = f"{exp_dir}/{exp_name}"

main_optuna_metric = "val_loss"
direction = "min"  # min or max

In [78]:
optuna_storage = init_optuna_storage()
old_study = optuna.load_study(study_name=study_name, storage=optuna_storage)
trials_info_df = pd.read_csv(str(os.path.join(exp_dir, exp_name, "trials_info.csv")), index_col=0)

exp_config = HTunerExpConfig.load_from_file(str(os.path.join(exp_dir, exp_name, HTUNER_CONFIG_FILE)))
exp_config.htuner

H:\programmazione\python_projects\ingredient_recognition\src\training\commons.py:74: ExperimentalWarning:

JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.



{'sampler': optuna.samplers._tpe.sampler.TPESampler,
 'sampler_kwargs': {'n_startup_trials': 7,
  'n_ei_candidates': 24,
  'multivariate': True,
  'group': True},
 'pruner': optuna.pruners._hyperband.HyperbandPruner,
 'pruner_kwargs': {'min_resource': 2,
  'max_resource': 'auto',
  'reduction_factor': 3},
 'direction': 'minimize',
 'n_trials': 100,
 'save_dir': 'H:\\programmazione\\python_projects\\ingredient_recognition\\experiments\\basic'}

In [79]:
trials_info_df = trials_info_df.rename(columns={"image_augmentation": "tm_trns_aug", "model_type": "tm_type"})
trials_info_df['tm_trns_aug'] = trials_info_df['tm_trns_aug'].apply(lambda x: "aug_hard" if x else "no_aug")

distributions = old_study.trials[-1].distributions
params_columns = list(distributions.keys())
states = [state for state in optuna.trial.TrialState]
states_map = {state.name: state for state in states}
states_map['ERROR'] = states_map['FAIL']  # ERROR is not a valid state, map it to FAIL

In [80]:
trials_info_df.columns

Index(['best_epoch', 'state', 'val_loss_max_diff%', 'hp_lr', 'hp_lr_scheduler',
       'hp_optimizer', 'hp_weight_decay', 'hp_weighted_loss', 'tm_trns_aug',
       'tm_type', 'val_acc', 'val_hamming', 'val_loss', 'val_precision',
       'val_recall', 'train_acc', 'train_hamming', 'train_loss_epoch',
       'train_loss_step', 'train_precision', 'train_recall'],
      dtype='object')

In [81]:
# #create a dict that maps the names in the dataframe to the optuna parameter names
# param_name_map = {
#     "lr": "hp_lr",
#     "lr_scheduler": "hp_lr_scheduler",
#     "optimizer": "hp_optimizer",
#     "weight_decay": "hp_weight_decay",
#     "weighted_loss": "hp_weighted_loss",
#     "tm_type": "tm_type",
#     "trns_aug": "tm_trns_aug"
# }

# distributions = {param_name_map[k]: v for k, v in distributions.items() if param_name_map[k] in params_columns}
# distributions


In [82]:
optuna_storage_trash = init_optuna_storage(OPTUNA_JOURNAL_TRASH_PATH)

htuner_config = exp_config.htuner
sampler = htuner_config["sampler"](**htuner_config["sampler_kwargs"])
pruner = htuner_config["pruner"](**htuner_config["pruner_kwargs"])

new_study = optuna.create_study(study_name=study_name, storage=optuna_storage_trash, sampler=sampler, pruner=pruner,
 direction=htuner_config['direction'], load_if_exists=True)



H:\programmazione\python_projects\ingredient_recognition\src\training\commons.py:74: ExperimentalWarning:

JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.

C:\Users\polil\.conda\envs\image_pytorch\lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning:

``multivariate`` option is an experimental feature. The interface can change in the future.

C:\Users\polil\.conda\envs\image_pytorch\lib\site-packages\optuna\samplers\_tpe\sampler.py:330: ExperimentalWarning:

``group`` option is an experimental feature. The interface can change in the future.

[I 2024-11-10 17:43:54,187] A new study created in Journal with name: H:\programmazione\python_projects\ingredient_recognition\experiments\basic/resnets_htuning{25k}


In [83]:
trials_info_df

,best_epoch,state,val_loss_max_diff%,hp_lr,hp_lr_scheduler,hp_optimizer,hp_weight_decay,hp_weighted_loss,tm_trns_aug,tm_type,...,val_hamming,val_loss,val_precision,val_recall,train_acc,train_hamming,train_loss_epoch,train_loss_step,train_precision,train_recall
trial,,,,,,,,,,,,,,,,,,,,,
0,33,COMPLETE,0.096854,0.002832,CosineAnnealingWarmRestarts,adam,0.001393,True,aug_hard,resnet18Pre,...,0.262115,0.952078,0.307288,0.475719,0.738374,0.261626,0.962378,0.970691,0.306431,0.450374
1,37,COMPLETE,0.022534,0.000180,none,adam,0.000126,True,aug_hard,dummy,...,0.223753,1.062193,0.143023,0.276554,0.775499,0.224501,1.060016,1.126330,0.144207,0.277445
2,39,COMPLETE,0.095076,0.148173,none,sgd,0.000013,False,no_aug,resnet18Like,...,0.187250,0.131263,0.333920,0.238216,0.812311,0.187688,0.129784,0.133762,0.334865,0.239137
3,6,PRUNED,0.005672,0.000013,ConstantStartReduceOnPlateau,sgd,0.088698,True,no_aug,resnet18Like,...,0.493576,1.173564,0.173406,0.546203,0.505615,0.494385,1.139694,1.106671,0.172945,0.542089
4,27,COMPLETE,0.288138,0.067416,none,sgd,0.055988,False,aug_hard,dummy,...,0.211100,0.146751,0.074827,0.127508,0.787241,0.212759,0.152431,0.154253,0.085904,0.149033
5,39,COMPLETE,0.605803,0.002405,ConstantStartReduceOnPlateau,sgd,0.000527,False,no_aug,dummy,...,0.203251,0.143111,0.156886,0.195854,0.795551,0.204449,0.142459,0.141271,0.164922,0.168825
6,2,PRUNED,0.206967,0.074914,CosineAnnealingWarmRestarts,adam,0.009892,False,aug_hard,resnet18Pre,...,0.216495,0.200774,0.042379,0.068007,0.785704,0.214296,0.197199,0.198966,0.075702,0.131526
7,2,PRUNED,0.256562,0.000992,CosineAnnealingWarmRestarts,sgd,0.005208,False,aug_hard,resnet18Like,...,0.238553,0.459206,0.139278,0.172688,0.696334,0.303666,0.560609,0.490573,0.164768,0.258191
8,0,ERROR,0.000000,0.212993,ConstantStartReduceOnPlateau,adam,0.000015,True,no_aug,resnet18Like,...,0.233246,110.000000,0.000000,0.000000,0.690094,0.309906,1.084960,1.113973,0.187277,0.338291


In [84]:
states_map

{'RUNNING': TrialState.RUNNING,
 'COMPLETE': TrialState.COMPLETE,
 'PRUNED': TrialState.PRUNED,
 'FAIL': TrialState.FAIL,
 'WAITING': TrialState.WAITING,
 'ERROR': TrialState.FAIL}

In [85]:
df = trials_info_df[params_columns + [main_optuna_metric, 'state']]
for row_name, row in df.iterrows():
    # Prepare trial parameters and result
    params = row[params_columns].to_dict()
    value = row[main_optuna_metric]

    # Create a FrozenTrial with a unique trial_id and your parameters
    trial = FrozenTrial(
        number=row_name,
        state=states_map[row['state']],
        value=value if row['state'] not in ["ERROR", "FAIL"] else None,
        datetime_start=datetime.datetime.now(),
        datetime_complete=datetime.datetime.now(),
        params=params,
        distributions=distributions,
        user_attrs={},
        system_attrs={},
        intermediate_values={},
        trial_id=row_name
    )
    new_study.add_trial(trial)


In [86]:
plot_param_importances(new_study).show()

In [75]:
importance = get_param_importances(new_study)
importance

{'hp_optimizer': 0.37562712489455524,
 'hp_weighted_loss': 0.3434056152744993,
 'hp_weight_decay': 0.13033070537797434,
 'tm_type': 0.06954311265458189,
 'hp_lr': 0.06267247328020364,
 'tm_trns_aug': 0.01786951288145894,
 'hp_lr_scheduler': 0.0005514556367265606}

In [87]:
study = new_study

from optuna.visualization import plot_rank, plot_contour, plot_parallel_coordinate

In [93]:
plot_rank(study).show(renderer="browser")

C:\Users\polil\AppData\Local\Temp\ipykernel_6436\1258363813.py:1: ExperimentalWarning:

plot_rank is experimental (supported from v3.2.0). The interface can change in the future.



In [94]:
plot_contour(study).show(renderer="browser")

In [99]:

plot_parallel_coordinate(study, params=list(study.best_params.keys()), target_name="val_loss").show(renderer="browser")



dict_keys(['hp_lr', 'hp_optimizer', 'hp_lr_scheduler', 'hp_weighted_loss', 'hp_weight_decay', 'tm_type', 'tm_trns_aug'])